In [ ]:
---
layout: post
title: Spring 5 Final Blog
description: Sprint 5 Blog
permalink: /sprint5/
comments: true
---

<style>
    body {
        background-color: #000000 !important;
        color: #ffffff !important;
        font-family: 'Courier New', Courier, monospace !important;
        line-height: 1.8 !important;
        margin: 0 !important;
        padding: 20px !important;
    }

    article {
        background: linear-gradient(145deg, #1e1e2f, #000000) !important;
        border: 4px solid #4a90e2 !important;
        padding: 25px !important;
        border-radius: 20px !important;
        box-shadow: 0 10px 20px rgba(0, 0, 0, 0.8) !important;
    }

    article h1 {
        text-align: center !important;
        font-size: 2.5em !important;
        color: #4a90e2 !important;
        text-shadow: 2px 2px 5px #000000 !important;
    }

    article ul {
        list-style-type: disc !important;
        margin: 15px 20px !important;
        padding-left: 40px !important;
    }

    article li {
        background-color: #2a2a40 !important;
        padding: 10px 15px !important;
        margin: 10px 0 !important;
        border-radius: 10px !important;
        border-left: 5px solid #4a90e2 !important;
        box-shadow: 0px 5px 10px rgba(74, 144, 226, 0.5) !important;
        color: #ffffff !important;
    }

    article code {
        display: block !important;
        background-color: #4a90e2 !important;
        color: #000000 !important;
        padding: 10px !important;
        margin: 15px 0 !important;
        border-radius: 8px !important;
        font-size: 1.2em !important;
    }

    article blockquote {
        font-style: italic !important;
        border-left: 5px solid #4a90e2 !important;
        margin: 20px 0 !important;
        padding-left: 20px !important;
        color: #d3d3d3 !important;
    }
</style>

<article>
    <h1>Free-Response Question Game Blog</h1>

<h2>1. Introduction</h2>
    <p>
        This blog showcases the technical aspects of the **Free-Response Question Game** application, focusing on the code behind its features and functionality.
    </p>

<h2>2. Frontend Features</h2>

<h3>Dynamic Question Generation</h3>
    <p>
        Users select a topic, and the app generates a thought-provoking question using the <strong>Gemini API</strong>.
    </p>
    <code>
        async function generateQuestion() {
            const topicSelect = document.getElementById('topicSelect');
            const questionBox = document.getElementById('questionBox');
            const selectedTopic = topicSelect.value;

            if (!selectedTopic) {
                questionBox.textContent = "Please select a topic first!";
                return;
            }

            questionBox.textContent = "Generating question...";

            const question = await sendToGeminiAPI(selectedTopic);
            questionBox.textContent = question;
        }
    </code>
 <p>
        This function fetches a question based on the selected topic and updates the DOM dynamically.
    </p>

 <h3>Group and Channel Selection</h3>
    <p>
        Groups and channels are dynamically loaded into dropdowns via API calls. Here's the code for fetching groups:
    </p>
    <code>
        async function fetchGroups() {
            try {
                const response = await fetch(`${pythonURI}/api/groups/filter`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ section_name: "Prism" })
                });
                if (!response.ok) throw new Error('Failed to fetch groups');
                const groups = await response.json();
                const groupSelect = document.getElementById('group_id');
                groups.forEach(group => {
                    const option = document.createElement('option');
                    option.value = group.name;
                    option.textContent = group.name;
                    groupSelect.appendChild(option);
                });
            } catch (error) {
                console.error('Error fetching groups:', error);
            }
        }
    </code>
    <p>
        This function fetches group data and populates the dropdown menu dynamically.
    </p>

<h3>Submitting Posts</h3>
    <p>
        Here's the code for handling post submissions from the frontend to the backend:
    </p>
    <code>
        document.getElementById('postForm').addEventListener('submit', async function(event) {
            event.preventDefault();
            const title = document.getElementById('title').value;
            const comment = document.getElementById('comment').value;
            const channelId = document.getElementById('channel_id').value;

            try {
                const response = await fetch(`${pythonURI}/api/post`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ title, comment, channel_id: channelId })
                });
                if (!response.ok) throw new Error('Failed to add post');
                const result = await response.json();
                alert(`Post added successfully! Post ID: ${result.id}`);
            } catch (error) {
                console.error('Error adding post:', error);
            }
        });
    </code>
 <p>
        This form submission handler sends post data to the backend and displays success or error messages.
    </p>

<h2>3. Backend Features</h2>

<h3>Post Creation Endpoint</h3>
    <p>
        The backend API allows users to create posts via the following endpoint:
    </p>
    <code>
        @token_required()
        def post(self):
            data = request.get_json()
            if 'title' not in data or 'comment' not in data or 'channel_id' not in data:
                return {'message': 'Missing required fields'}, 400
            post = Post(data['title'], data['comment'], g.current_user.id, data['channel_id'])
            post.create()
            return jsonify(post.read())
    </code>
    <p>
        This function validates the input, creates a new post, and saves it to the database.
    </p>

 <h3>Post Retrieval Endpoint</h3>
    <p>
        The API retrieves posts based on filters such as channel ID:
    </p>
    <code>
        @token_required()
        def post(self):
            data = request.get_json()
            if 'channel_id' not in data:
                return {'message': 'Channel ID not provided'}, 400
            posts = Post.query.filter_by(_channel_id=data['channel_id']).all()
            return jsonify([post.read() for post in posts])
    </code>
    <p>
        This endpoint retrieves all posts for a specific channel, returning them in JSON format.
    </p>

<h3>Database Initialization</h3>
    <p>
        The database is initialized with tester data using the following function:
    </p>
    <code>
        def initPosts():
            posts = [
                Post(title='Sample Post 1', comment='This is a test post.', user_id=1, channel_id=1),
                Post(title='Sample Post 2', comment='Another test post.', user_id=2, channel_id=1)
            ]
            for post in posts:
                try:
                    post.create()
                except IntegrityError:
                    db.session.rollback()
    </code>
    <p>
        This function preloads the database with sample posts for testing purposes.
    </p>

<h2>4. API Communication</h2>
    <p>
        Frontend and backend communicate via RESTful principles, using endpoints such as:
    </p>
    <code>
        POST /api/post        # Create a post
        GET /api/posts/filter # Retrieve posts for a specific channel
        PUT /api/post         # Update a post
        DELETE /api/post      # Delete a post
    </code>
    <p>
        The frontend uses <code>fetch()</code> to send JSON payloads and handle responses, as shown below:
    </p>
    <code>
        const response = await fetch(`${pythonURI}/api/post`, {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify(postData)
        });
    </code>

<h2>5. Challenges and Improvements</h2>
    <p>
        <strong>Challenges:</strong>
    </p>
    <ul>
        <li>Debugging API responses and ensuring data validation.</li>
        <li>Designing a frontend that integrates seamlessly with backend APIs.</li>
    </ul>
    <h2>6. Addressing Learning Requirements</h2>
<p>
    This blog aligns with the outlined learning requirements by demonstrating the following:
</p>
<ul>
    <li><strong>Programming as a collaborative and creative process:</strong> The project integrates both frontend and backend components, requiring collaboration and creative problem-solving.</li>
    <li><strong>Group and individual contributions:</strong> The blog highlights the group’s purpose (creating a free-response question platform) and my individual contributions (frontend features, API integration, and CRUD endpoints).</li>
    <li><strong>Input/Output demonstration:</strong> Examples of user inputs (topic selection, form submissions) and outputs (API responses, rendered DOM updates) are included.</li>
    <li><strong>API request handling:</strong> Postman examples and frontend fetch requests showcase how inputs are sent and processed via RESTful APIs.</li>
    <li><strong>Database management:</strong> Demonstrates initializing, querying, and restoring tester data using SQLAlchemy.</li>
    <li><strong>Algorithmic logic:</strong> CRUD operations and frontend request/response handling show sequencing, selection, and iteration.</li>
    <li><strong>Error handling:</strong> Highlights how invalid inputs trigger different backend responses.</li>
</ul>
<p>
    By addressing these requirements, the blog provides a complete overview of the technical implementation and learning outcomes of the project.
</p>

<h3>Third-Party Libraries</h3>
<p>
    This project uses <strong>SQLAlchemy</strong> as a third-party library for Object-Relational Mapping (ORM). SQLAlchemy simplifies database operations, including querying rows (lists) and managing table columns (dictionaries). Its integration ensures data consistency and reduces boilerplate code, allowing for more efficient database management.
</p>

<p>
        <strong>Improvements:</strong>
    </p>
    <ul>
        <li>Add better error messages for invalid inputs.</li>
        <li>Optimize database queries for large datasets.</li>
    </ul>

<h2>Conclusion</h2>
    <p>
        This blog highlights the technical aspects of the Free-Response Question Game, focusing on key code snippets and their explanations. The project demonstrates the effective use of Flask, SQLAlchemy, and modern JavaScript for building a robust full-stack application.
    </p>
</article>
